In [1]:
import os
import json
import requests
import time
from bs4 import BeautifulSoup

In [26]:
db.query("select * from tag")

({'id': 111, 'name': '0_1_bfs'},
 {'id': 87, 'name': '2_sat'},
 {'id': 175, 'name': 'a_star'},
 {'id': 23, 'name': 'ad_hoc'},
 {'id': 113, 'name': 'aho_corasick'},
 {'id': 135, 'name': 'alien'},
 {'id': 45, 'name': 'arbitrary_precision'},
 {'id': 17, 'name': 'arithmetic'},
 {'id': 86, 'name': 'articulation'},
 {'id': 29, 'name': 'backtracking'},
 {'id': 169, 'name': 'bayes'},
 {'id': 115, 'name': 'bellman_ford'},
 {'id': 109, 'name': 'berlekamp_massey'},
 {'id': 16, 'name': 'bfs'},
 {'id': 107, 'name': 'biconnected_component'},
 {'id': 158, 'name': 'bidirectional_search'},
 {'id': 15, 'name': 'binary_search'},
 {'id': 49, 'name': 'bipartite_matching'},
 {'id': 24, 'name': 'bitmask'},
 {'id': 93, 'name': 'bitset'},
 {'id': 9, 'name': 'bruteforcing'},
 {'id': 145, 'name': 'burnside'},
 {'id': 96, 'name': 'cactus'},
 {'id': 75, 'name': 'calculus'},
 {'id': 26, 'name': 'case_work'},
 {'id': 83, 'name': 'centroid'},
 {'id': 84, 'name': 'centroid_decomposition'},
 {'id': 69, 'name': 'cht'},


In [28]:
db.query("select * from user where solved is null limit 10")

({'id': 298, 'handle': 'qudgml6323', 'solved': None, 'submit': None},
 {'id': 451, 'handle': 'white_mocha', 'solved': None, 'submit': None},
 {'id': 510, 'handle': 'jjaewon', 'solved': None, 'submit': None},
 {'id': 526, 'handle': 'taehoon1310', 'solved': None, 'submit': None},
 {'id': 743, 'handle': 'ronaldo', 'solved': None, 'submit': None},
 {'id': 1068, 'handle': 'auddl0756', 'solved': None, 'submit': None},
 {'id': 1225, 'handle': 'riew610', 'solved': None, 'submit': None},
 {'id': 1363, 'handle': 'bjm7701', 'solved': None, 'submit': None},
 {'id': 1385, 'handle': 'jonahan', 'solved': None, 'submit': None},
 {'id': 1463, 'handle': 'undefined_y', 'solved': None, 'submit': None})

In [6]:
solved_lookup = "https://solved.ac/api/v3/problem/lookup"

In [92]:
res=db.query("select id from problem")

In [7]:
r = requests.get("https://solved.ac/api/v3/problem/lookup?problemIds=1000%2C1001%2C5000")
print(r.text)

[{"problemId":1000,"titleKo":"A+B","isSolvable":true,"isPartial":false,"acceptedUserCount":151801,"level":1,"votedUserCount":17,"isLevelLocked":true,"averageTries":2.337,"official":true,"tags":[{"key":"arithmetic","isMeta":false,"bojTagId":121,"problemCount":495,"displayNames":[{"language":"en","name":"arithmetic","short":"arithmetic"},{"language":"ko","name":"사칙연산","short":"사칙연산"}]},{"key":"implementation","isMeta":false,"bojTagId":102,"problemCount":3089,"displayNames":[{"language":"en","name":"implementation","short":"impl"},{"language":"ko","name":"구현","short":"구현"}]},{"key":"math","isMeta":false,"bojTagId":124,"problemCount":3394,"displayNames":[{"language":"en","name":"mathematics","short":"math"},{"language":"ko","name":"수학","short":"수학"}]}]},{"problemId":1001,"titleKo":"A-B","isSolvable":true,"isPartial":false,"acceptedUserCount":126316,"level":1,"votedUserCount":8,"isLevelLocked":true,"averageTries":1.3909,"official":true,"tags":[{"key":"arithmetic","isMeta":false,"bojTagId":1

In [52]:
problem_id = 1000
query_string = f'\
                SELECT tag.name \
                FROM tag join (\
                    SELECT tag_id \
                    FROM problem join problem_tag on problem.id = problem_tag.problem_id \
                    WHERE problem.id = {problem_id}) as t1 on tag.id = t1.tag_id'
print([row['name'] for row in db.query(query_string)])

['math', 'implementation', 'arithmetic']


In [51]:
res = db.query('select user_id, problem_id, count(user_id) from ac group by user_id, problem_id having count(user_id) > 1')
for row in res:
    problem_id = row['problem_id']
    user_id = row['user_id']
    db.query(f'delete from ac where user_id={user_id} and problem_id={problem_id} limit 1')

In [25]:
tier_to_exp = dict()


def initialize():
    res = db.query('select * from tier_exp')
    for row in res:
        tier_to_exp[row['tier']] = row['exp']
initialize()

def update_problems():
    # can get problem by id array with query param at most 100
    solved_url = "https://solved.ac"
    start_num = 1000
    last_num = 23000
    lookup_url = solved_url + "/api/v3/problem/lookup"
    batch_size = 1
    for pid in range(start_num, last_num+1, batch_size):
        id_list = ",".join([str(i) for i in range(pid, pid+batch_size)])
        payload = {"problemIds":id_list}
        r = requests.get(lookup_url, params=payload)
        for problem in r.json():
            problem_id = problem['problemId']
            name = problem['titleKo']
            tier = problem['level']
            exp = tier_to_exp[tier]
            query_string = f'INSERT INTO problem (id, tier, exp, name) \
                                    VALUES ({problem_id}, {tier}, {exp}, \'{name}\') \
                                    ON DUPLICATE KEY UPDATE \
                                    tier={tier}, \
                                    exp={exp}, \
                                    name=\'{name}\''
            print(query_string)
        break
update_problems()

INSERT INTO problem (id, tier, exp, name)                                     VALUES (5000, 19, 630792, '빵 정렬')                                     ON DUPLICATE KEY UPDATE                                     tier=19,                                     exp=630792,                                     name='빵 정렬'


In [110]:
print(payload)

{'problemIds': '1000%2C1001%2C1002%2C1003%2C1004%2C1005%2C1006%2C1007%2C1008%2C1009'}


In [15]:
str_int = [str(row) for row in range(11100, 11110)]
payload = {"problemIds":",".join(str_int)}
resp = requests.get(solved_lookup, params=payload)

In [16]:
print(resp.url)
print(resp.status_code)
print(json.dumps(resp.json(), indent=4, ensure_ascii=False))

https://solved.ac/api/v3/problem/lookup?problemIds=11100%2C11101%2C11102%2C11103%2C11104%2C11105%2C11106%2C11107%2C11108%2C11109
200
[
    {
        "problemId": 11100,
        "titleKo": "Frogger",
        "isSolvable": true,
        "isPartial": false,
        "acceptedUserCount": 33,
        "level": 0,
        "votedUserCount": 0,
        "isLevelLocked": false,
        "averageTries": 3.3636,
        "official": true,
        "tags": []
    },
    {
        "problemId": 11101,
        "titleKo": "꿍의 여친 만들기",
        "isSolvable": true,
        "isPartial": false,
        "acceptedUserCount": 33,
        "level": 0,
        "votedUserCount": 0,
        "isLevelLocked": false,
        "averageTries": 1.4545,
        "official": true,
        "tags": []
    },
    {
        "problemId": 11102,
        "titleKo": "Party",
        "isSolvable": true,
        "isPartial": false,
        "acceptedUserCount": 18,
        "level": 18,
        "votedUserCount": 3,
        "isLevelLocked": f

In [2]:
from utils import DBHelper
db = DBHelper('.mylogin.cnf')

In [8]:
rank_url="https://www.acmicpc.net/ranklist"
resp = requests.get(rank_url)

In [146]:
url = "https://www.acmicpc.net/problemset?sort=no_desc"
r = requests.get(url)
soup = BeautifulSoup(r.text)
last_number = int(soup.find('table', id='problemset').find('tbody').find('tr').find('td').text)
print(last_number)


23824


In [65]:
for page_num in range(401, 601):
    url = rank_url + f'/{page_num}'
    print(url)
    resp = requests.get(url)
    soup = BeautifulSoup(resp.text)
    tables = soup.find('table', id='ranklist')
    rows = tables.find_all('tr')
    for row in rows[1:]:
        info = [elem.text for elem in row.find_all('td')]
        user_info.append(" ".join([info[1], info[3], info[4]]))
    time.sleep(0.5)

https://www.acmicpc.net/ranklist/401
https://www.acmicpc.net/ranklist/402
https://www.acmicpc.net/ranklist/403
https://www.acmicpc.net/ranklist/404
https://www.acmicpc.net/ranklist/405
https://www.acmicpc.net/ranklist/406
https://www.acmicpc.net/ranklist/407
https://www.acmicpc.net/ranklist/408
https://www.acmicpc.net/ranklist/409
https://www.acmicpc.net/ranklist/410
https://www.acmicpc.net/ranklist/411
https://www.acmicpc.net/ranklist/412
https://www.acmicpc.net/ranklist/413
https://www.acmicpc.net/ranklist/414
https://www.acmicpc.net/ranklist/415
https://www.acmicpc.net/ranklist/416
https://www.acmicpc.net/ranklist/417
https://www.acmicpc.net/ranklist/418
https://www.acmicpc.net/ranklist/419
https://www.acmicpc.net/ranklist/420
https://www.acmicpc.net/ranklist/421
https://www.acmicpc.net/ranklist/422
https://www.acmicpc.net/ranklist/423
https://www.acmicpc.net/ranklist/424
https://www.acmicpc.net/ranklist/425
https://www.acmicpc.net/ranklist/426
https://www.acmicpc.net/ranklist/427
h

In [66]:
with open("user_info", "w") as f:
    f.write("\n".join(user_info))

In [67]:
print(len(user_info))
for info in user_info:
    handle, solved, submit = info.split()
    solved = int(solved)
    submit = int(submit)
    query_string = ''
    if db.check_user(handle):
        query_string = f'update user set solved={solved}, submit={submit} where handle=\'{handle}\''
        db.query(query_string)
    else:
        query_string = f'insert into user(handle, solved, submit) values (\'{handle}\', {solved}, {submit})'
        db.query(query_string)
db.connection.commit()

60000


In [27]:
soup = BeautifulSoup(resp.text)
tables = soup.find('table', id='ranklist')
rows = tables.find_all('tr')
for row in rows[1:]:
    for elem in row.find_all('td'):
        print(elem.text)
    break

1
koosaga
for(int i = x; i < MAXN; i += i & -i) tree[i] += v;
10543
21446
56.178%


In [71]:
base_url = "https://www.acmicpc.net/user/"
user_directory = "solved_data/user"
solved_directory = "solved_data/user_solved"
sleep_seconds = 30

In [49]:
def call_file_sorted(path):
    directory = os.fsencode(path)
    ret = []
    for file in os.listdir(directory):
        filename = os.fsdecode(file)
        filename = os.path.join(path, filename)
        ret.append(filename)
    ret.sort()
    return ret

def change_path(target_path, source_path, ext):
    basename = os.path.basename(source_path)
    basename = os.path.splitext(basename)[0]
    return_path = os.path.join(target_path, basename) + ext
    return return_path

In [73]:
def get_problem_list(handle):
    url = base_url + handle
    r = requests.get(url)
    if r.status_code == 404:
        print(handle)
        return [-1]
    if r.status_code != 200:
        print(r.text)
        print(handle)
        time.sleep(sleep_seconds)
        return get_problem_list(handle)
    soup = BeautifulSoup(r.text, "html.parser")
    ret = []
    for div in soup.find_all("div", {"class":"panel-body"})[0]:
        if div.find("a") is None:
            ret.append(div.text)
    return ret

In [46]:
handle = "duram21"
resp = requests.get("https://www.acmicpc.net/user/" + handle)

In [47]:
soup = BeautifulSoup(resp.text, features='html.parser')
ac_list = list()
for ac in soup.find("div", {"class": "problem-list"}).find_all('a'):
    ac_list.append(int(ac.text))
res = soup.find('table', {"id": "statics"}).find_all('tr')
solved, submit = res[1].find('td').text, res[3].find('td').text
print(solved, submit)
print(ac_list)

852 37
[1000, 1001, 1003, 1005, 1008, 1009, 1010, 1012, 1015, 1016, 1018, 1021, 1024, 1026, 1032, 1037, 1038, 1041, 1049, 1051, 1065, 1068, 1074, 1075, 1076, 1083, 1085, 1092, 1094, 1100, 1106, 1110, 1120, 1124, 1132, 1135, 1138, 1145, 1149, 1152, 1157, 1158, 1159, 1167, 1173, 1181, 1182, 1193, 1197, 1202, 1205, 1212, 1213, 1225, 1233, 1235, 1238, 1240, 1244, 1246, 1247, 1252, 1253, 1259, 1260, 1264, 1267, 1269, 1271, 1275, 1280, 1284, 1292, 1296, 1297, 1300, 1306, 1309, 1316, 1323, 1325, 1330, 1334, 1339, 1340, 1343, 1350, 1351, 1356, 1357, 1359, 1365, 1374, 1377, 1406, 1408, 1422, 1427, 1436, 1437, 1439, 1446, 1448, 1449, 1453, 1461, 1463, 1475, 1476, 1484, 1487, 1490, 1503, 1504, 1516, 1520, 1526, 1541, 1543, 1546, 1547, 1550, 1551, 1564, 1568, 1572, 1592, 1598, 1600, 1615, 1620, 1644, 1647, 1654, 1655, 1668, 1673, 1676, 1681, 1689, 1695, 1697, 1699, 1707, 1712, 1715, 1717, 1725, 1731, 1735, 1737, 1744, 1747, 1748, 1753, 1755, 1758, 1759, 1764, 1766, 1769, 1773, 1781, 1783, 1788, 17

In [64]:
user_json_list = call_file_sorted(user_directory)
cnt = 0
for user_json in user_json_list:
    time.sleep(1)
    with open(user_json, "r", encoding="utf-8") as f:
        handle = json.load(f)['handle']
    p_list = get_problem_list(handle)
    solved_path = change_path(solved_directory, user_json, ".json")
    res = {'handle':handle, 'count':len(p_list), 'problems':p_list}
    with open(solved_path, "w", encoding="utf-8") as f:
        json.dump(res, f, ensure_ascii=False)
    cnt += 1
    if cnt % 50 == 0:
        print(cnt, "Done")

50 Done
100 Done
150 Done
200 Done
250 Done
300 Done
350 Done
400 Done
450 Done
white_mocha
500 Done
jjaewon
taehoon1310
550 Done
600 Done
650 Done
700 Done
ronaldo
750 Done
800 Done
850 Done
900 Done
950 Done
1000 Done
1050 Done
auddl0756
1100 Done
1150 Done
1200 Done
riew610
1250 Done
1300 Done
1350 Done
jonahan
1400 Done
1450 Done
undefined_y
1500 Done
chlwndud2347
1550 Done
97k07h11
1600 Done
sojindh
1650 Done
avengers
1700 Done
1750 Done
1800 Done
1850 Done
marshmelloooooooooow
1900 Done
1950 Done
2000 Done
2050 Done
2100 Done
honeymint31
2150 Done
2200 Done
2250 Done
2300 Done
2350 Done
2400 Done
star0202_dev
2450 Done
nonassembly
2500 Done
2550 Done
2600 Done
2650 Done
2700 Done
2750 Done
2800 Done
2850 Done
2900 Done
2950 Done
3000 Done
3050 Done
3100 Done
3150 Done
3200 Done
3250 Done
3300 Done
yopp26
3350 Done
nmnm106803
3400 Done
3450 Done
3500 Done
3550 Done
3600 Done
3650 Done
3700 Done
3750 Done
3800 Done
3850 Done
3900 Done
3950 Done
4000 Done
4050 Done
4100 Done
4150 Do

rootpi
31200 Done
ekrtjsdl
31250 Done
ljy325732
kkkddyyyyy
31300 Done
pysun
31350 Done
31400 Done
31450 Done
skybluelab
hi_jihyo
31500 Done
31550 Done
31600 Done
31650 Done
31700 Done
31750 Done
31800 Done
31850 Done
madcho
topaz6623
31900 Done
jonahan128
31950 Done
dawon323
32000 Done
32050 Done
32100 Done
32150 Done
hasaja
32200 Done
cewjd76
32250 Done
koko1101
32300 Done
32350 Done
32400 Done
32450 Done
32500 Done
32550 Done
32600 Done
juyeon1317
32650 Done
32700 Done
32750 Done
32800 Done
32850 Done
butterflo
32900 Done
32950 Done
33000 Done
sogangjason
33050 Done
33100 Done
33150 Done
33200 Done
33250 Done
csm1959
33300 Done
psydangi02
hhhan0315
33350 Done
jjwon4086
33400 Done
imeubug
33450 Done
33500 Done
33550 Done
gmldud8313
33600 Done
winawer
33650 Done
33700 Done
33750 Done
shmom467
33800 Done
33850 Done
33900 Done
33950 Done
fhdsvdjls
geolyun
kdo9921
34000 Done
34050 Done
migeng8020
34100 Done
34150 Done
34200 Done
chanel
34250 Done
jhhan128
34300 Done
34350 Done
flowcup
344

In [52]:
print(cnt)

450
